# Atomap tutorial: finding and analysing sublattices

For more details see the open access article: **Atomap: a new software tool for the automated analysis of atomic resolution images using two-dimensional Gaussian fitting**. https://dx.doi.org/10.1186/s40679-017-0042-5

This tutorial shows how to use Atomap to analyse the atomic structure of a perovskite oxide structure, by manually finding the two sublattices in an image.

More documentation is found at http://atomap.org/

## Importing the libraries

Firstly, we must set the plotting toolkit:

In [ ]:
%matplotlib notebook

Atomap relies heavily on HyperSpy for visualization and fitting, and uses HyperSpy signals for most of the outputs. So we need to import both HyperSpy and Atomap.

You might get a "WARNING:hyperspy_gui_traitsui", this can be ignored.

In [ ]:
import hyperspy.api as hs

In [ ]:
import atomap.api as am

### Loading data

Atomap uses HyperSpy signals as its input, which can be any loaded from many different types of files. DM3/DM4, tif, emi/ser, jpg or HDF5-files. This can be loaded using `s = hs.load(your_filename)`.

Here we will be using a test dataset, generated by the `dummy_data` module.

In [ ]:
s = am.dummy_data.get_fantasite()

This is an imaginary structure, similar to a perovskite oxide projected along on the [110] direction.

In [ ]:
s.plot()

## Finding initial positions

Our first task is finding and fitting the most intense atomic columns. This is done by using a peak finder, where the minimum distance between the features is used as an input parameter. This function returns a HyperSpy signal, where the atom positions are saved as permanent markers in the metadata

In [ ]:
s_peaks = am.get_feature_separation(s, pca=True)

In this plot, the minimum feature separation is shown on the x-axis for the navigation plot. This parameter can be changed by using the arrow keys. For this image, we want to get the peaks for only the most intense atomic columns.
A feature separation of 12 works fine.

In [ ]:
s_peaks.plot()

We use this value as an input for the next step, which involves getting these atomic positions as a list.

In [ ]:
atom_positions = am.get_atom_positions(s, separation=12, pca=True)

To add or remove atoms, use `add_atoms_with_gui`. Any changes will automatically be updated in the output list `atom_positions_manual`.

In [ ]:
atom_positions_manual = am.add_atoms_with_gui(s, atom_positions)

## Making a sublattice

The atom positions are used to initialize the sublattice object.

In [ ]:
sublattice = am.Sublattice(atom_position_list=atom_positions_manual, image=s.data)

This sublattice object contains the atom positions we just found, the image, and many utility functions. For example a plot function.

In [ ]:
sublattice.plot()

The first step is to refine the atomic positions using center of mass, since the initial atom positions are not very good.

In [ ]:
sublattice.find_nearest_neighbors()
sublattice.refine_atom_positions_using_center_of_mass()

Then using 2D Gaussians

In [ ]:
sublattice.refine_atom_positions_using_2d_gaussian()

This will give much more accurate atomic positions:

In [ ]:
sublattice.plot()

## Constructing zone axes

The next step is to find the relation between the atom positions, by ordering them into atomic planes:

In [ ]:
sublattice.construct_zone_axes()

These atomic planes can be visualized. Navigate using the arrow keys. The number to the top left is the direction of the zone axis.

In [ ]:
sublattice.plot_planes()

## Finding the second sublattice

In this structure, we have two separate sublattices. Having found the first one, we can use those atomic positions to find the second one. Here we use the fact that the atoms in the second sublattice is located between the atoms in the second zone axis for the first sublattice.

In [ ]:
zone_axis_001 = sublattice.zones_axis_average_distances[1]
zone_axis_001

Then we pass this to the `find_missing_atoms_from_zone_vector`, which returns a new list of positions

In [ ]:
positions2 = sublattice.find_missing_atoms_from_zone_vector(zone_axis_001)

Secondly, we "subtract" the intensity of the `sublattice` atoms from the fantasite image

In [ ]:
from atomap.tools import remove_atoms_from_image_using_2d_gaussian
image_atoms_subtracted = remove_atoms_from_image_using_2d_gaussian(sublattice.image, sublattice)

Which we use to make the second sublattice:

In [ ]:
sublattice2 = am.Sublattice(positions2, image_atoms_subtracted, color='blue')
sublattice2.plot()

And refine the positions

In [ ]:
sublattice2.construct_zone_axes()
sublattice2.refine_atom_positions_using_center_of_mass()
sublattice2.refine_atom_positions_using_2d_gaussian()

## The atom lattice object

Having made these two sublattices, we can combine them in an `Atom_Lattice` object

In [ ]:
atom_lattice = am.Atom_Lattice(image=s.data, name='fantasite', sublattice_list=[sublattice, sublattice2])
atom_lattice.plot()

Atom_Lattice objects can be stored, to avoid having to run the often slow fitting routines several times.

In [ ]:
atom_lattice.save(overwrite=True)

Which can be restored:

In [ ]:
atom_lattice2 = am.load_atom_lattice_from_hdf5("fantasite_atom_lattice.hdf5")

## Analysing the structure

The sublattice object contains many utility functions for visualizing various structural effects.

For example, the distance between the atoms.

In [ ]:
s_monolayer = sublattice2.get_monolayer_distance_map()
s_monolayer.plot()

Or the variations in distance between the atoms:

In [ ]:
s_distance_difference = sublattice2.get_atom_distance_difference_map()
s_distance_difference.plot()

Or the ellipticity of the atoms, which can be used to extract information about the structure parallel to the electron beam. Plotting only the magnitude:

In [ ]:
s_elli = sublattice.get_ellipticity_map()
s_elli.plot()

Plotting both the magnitude, and the direction:

In [ ]:
sublattice.plot_ellipticity_vectors()

## Integrating intensity

The intensity of the atomic columns can be found using `integrate_column_intensity` function, which supports two different methods: `Watershed` and `Voronoi`, where the latter is the default one.

In [ ]:
i_points, i_record, p_record = atom_lattice.integrate_column_intensity()
i_record.plot()

This also works for sublattices directly. Since the fantasite dataset contain several sublattices, the results here will contain intensity from several columns. So for situation like this, the `atom_lattice.integrate_column_intensity` should be used.

In [ ]:
i_points, i_record, p_record = sublattice.integrate_column_intensity()
i_record.plot()

## Line profiles

All these structural properties can also be plotted as line profiles, as a function of distance from some atomic plane. For example the distance between the atoms in the horizontal direction.

This requires two things: 1) the zone axis perpendicular to the horizontal direction, and 2) some atomic plane perpendicular to the horizontal direction.

The former is found using `plot_planes`

In [ ]:
sublattice2.plot_planes()

Which shows that the second zone axis is the one perpendicular to the horisontal axis. Which we get from `zones_axis_average_distances`. Note that Python is zero-indexed, meaning we need to get index `1`

In [ ]:
zone = sublattice2.zones_axis_average_distances[1]

Then we extract an atomic plane which is also perpendicular to the horizontal axis

In [ ]:
plane = sublattice2.atom_planes_by_zone_vector[zone][10]

Lastly, we get the line profile:

In [ ]:
s_monolayer_line = sublattice2.get_monolayer_distance_line_profile(zone_vector=zone, atom_plane=plane)
s_monolayer_line.plot()

This can be done for other properties as well:

In [ ]:
s_elli_line = sublattice.get_ellipticity_line_profile(atom_plane=plane)
s_elli_line.plot()

## Properties

The atomic positions can be accessed directly as well

In [ ]:
sublattice.x_position

These properites include: `y_position`, `sigma_x`, `sigma_y`, `ellipticity`, `rotation_ellipticity`.

These can be visualized using a variety of methods in the `sublattice` object, or stored to (for example) a csv-file which can be opened in spreadsheet software

In [ ]:
import numpy as np
np.savetxt("datafile.csv", (sublattice.x_position,
                            sublattice.y_position,
                            sublattice.sigma_x,
                            sublattice.sigma_y,
                            sublattice.ellipticity),
           delimiter=',')

The easiest way to save the monolayer map from earlier is through the HyperSpy HDF5 format

In [ ]:
s_monolayer.inav[0].save("monolayer_distance_map0.hdf5", overwrite=True)

These signals can also be saved as tif, so they can be opened in other image viewers. For this to work, we must first change NaN values to zero:

In [ ]:
import numpy as np
s_monolayer.data = np.nan_to_num(s_monolayer.data)

This file can be opened in image viewers such as ImageJ or Digital Micrograph

In [ ]:
s_monolayer.save("monolayer_distance_map0.tif", overwrite=True)

## Determining polarization

In many ferroelectric materials, the spontaneous electric polarization can be determined by looking at the shift of some atomic columns in relation to the others. One example of this is in the ferroelectric perovskite oxides, where the B-cation is shifted from its cubic centrosymmetric position. The polarization can be determined by finding both the direction and magnitude of this shift.

Firstly, we get an appropriate artificial dataset, resembling a ferroelectric thin film grown on top of a non-ferroelectric substrate.

In [ ]:
atom_lattice = am.dummy_data.get_polarization_film_atom_lattice()
atom_lattice.plot()

The blue, B-cation, atom columns in the top part of the image are shifted towards the left. By finding the centre position of four neighboring red A-cation forming a square, this shift can be quantified.

Finding these neighbors relies on moving along two zone axis directions in the A-cation sublattice.

In [ ]:
sublatticeA = atom_lattice.sublattice_list[0]
sublatticeA.construct_zone_axes()

Next, find the two perpendicular zone axes spanning this square. For the perovskite oxide 100 projection, this is most likely the two first ones.

In [ ]:
sublatticeA.plot_planes()

The zone axes are then used with the B-cation sublattice:

In [ ]:
za0 = sublatticeA.zones_axis_average_distances[0]
za1 = sublatticeA.zones_axis_average_distances[1]
sublatticeB = atom_lattice.sublattice_list[1]
s_polarization = sublatticeA.get_polarization_from_second_sublattice(za0, za1, sublatticeB)

This can be visualized directly by using the ``plot`` method, and the data itself can be accessed in the signal’s metadata.

In [ ]:
s_polarization.plot()
vector_list = s_polarization.metadata.vector_list